In [1]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ultralytics deep-sort-realtime opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# Loading YOLOv8 model
model = YOLO("yolov8n.pt")

# Define vehicle classes (COCO dataset IDs for vehicles)
vehicle_classes = [2, 3, 5, 7]  # Car, Bus, Truck, Motorcycle

# Variables for waiting zone
waiting_zone = []
zone_defined = False  # Flag to check if the zone is defined


def mouse_callback(event, x, y, flags, param):
    """
    Mouse callback to define the waiting zone by clicking.
    """
    global waiting_zone, zone_defined
    if event == cv2.EVENT_LBUTTONDOWN and not zone_defined:
        if len(waiting_zone) < 4:
            waiting_zone.append((x, y))
            print(f"Point {len(waiting_zone)} added: {x}, {y}")
        if len(waiting_zone) == 4:
            print("Waiting zone defined.")
            zone_defined = True


# Opening video stream
cap = cv2.VideoCapture("Lane1.mp4")

if not cap.isOpened():
    print("Error: Unable to open video source.")
    exit()

# Creating a named window and set the mouse callback
cv2.namedWindow("Traffic Waiting Zone")
cv2.setMouseCallback("Traffic Waiting Zone", mouse_callback)

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video stream or cannot read frame.")
        break

    # Showing frame for defining waiting zone
    if not zone_defined:
        for point in waiting_zone:
            cv2.circle(frame, point, 5, (0, 0, 255), -1)
        if len(waiting_zone) > 1:
            cv2.polylines(frame, [np.array(waiting_zone)], isClosed=False, color=(0, 0, 255), thickness=2)
        cv2.imshow("Traffic Waiting Zone", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    # Draw the waiting zone
    if zone_defined and len(waiting_zone) == 4:
        cv2.polylines(frame, [np.array(waiting_zone)], isClosed=True, color=(0, 255, 0), thickness=2)

    # Run YOLOv8 inference
    results = model(frame, stream=False, verbose=False)

    # Reset vehicle count for each frame
    vehicle_count = 0

    # Loop through detections
    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls.cpu().numpy().item())
            if cls_id in vehicle_classes:  # Checking if it's a vehicle
                x, y, w, h = map(int, box.xywh.cpu().numpy()[0])
                x1, y1 = x - w // 2, y - h // 2
                x2, y2 = x + w // 2, y + h // 2

                # Checking if the vehicle is in the waiting zone
                if zone_defined and cv2.pointPolygonTest(np.array(waiting_zone), (x, y), False) >= 0:
                    vehicle_count += 1

                # Drawing the bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, "Vehicle", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (0, 255, 255), 2)

    # Displaying vehicle count
    cv2.putText(frame, f"Waiting Vehicles: {vehicle_count}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 0, 0), 2)

    # Showing the frame
    cv2.imshow("Traffic Waiting Zone", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
'''Without Ambulance'''
import cv2
from ultralytics import YOLO
import numpy as np
import time
import copy
import threading

# Loading YOLOv8 model
model = YOLO("yolov8n.pt") 

#Loading ambulance trained model 
ambulance_model = YOLO("C:/Users/ragha/runs/detect/train5/weights/best.pt")

# Defining vehicle classes (COCO dataset IDs for vehicles)
vehicle_classes = [2, 3, 5, 7]  # Car, Bus, Truck, Motorcycle

# Initializing variables for lane monitoring
lanes = {
    "lane_1": {"cctv": "Lane1.mp4", "count": 0, "ambulance_detected": False},
    "lane_2": {"cctv": "Lane2.mp4", "count": 0, "ambulance_detected": False},
    "lane_3": {"cctv": "Lane3.mp4", "count": 0, "ambulance_detected": False},
    "lane_4": {"cctv": "Lane4.mp4", "count": 0, "ambulance_detected": False}
}

# Signal states and timers
signal_states = {"lane_1": "red", "lane_2": "red", "lane_3": "red", "lane_4": "red"}
signal_timers = {"lane_1": 0, "lane_2": 0, "lane_3": 0, "lane_4": 0}

# Flag to stop the process
stop_process = False
lock = threading.Lock()


caps = {lane_key: cv2.VideoCapture(lanes[lane_key]["cctv"]) for lane_key in lanes.keys()}
for cap in caps.values():
        cap.set(cv2.CAP_PROP_FPS, 1200)

def count_vehicles_and_draw(frame, lane_key):
    """
    Detect and count vehicles in the frame using YOLO, and draw bounding boxes.
    """
    results = model(frame, stream=False, verbose=False)
    vehicle_count = 0

    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls.cpu().numpy().item())
            if cls_id in vehicle_classes:  # Checking if the detected object is a vehicle
                vehicle_count += 1

                # Drawing bounding box
                x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    frame,
                    model.names[cls_id],
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    2,
                )

    with lock:
        lanes[lane_key]["count"] = vehicle_count
    return frame


def process_lane():
    """
    Process each lane's CCTV feed and update vehicle counts with detection frames.
    """
    for lane_key in ['lane_1','lane_2','lane_3', 'lane_4']:
        cap =cv2.VideoCapture(lanes[lane_key]["cctv"])
        ret, frame = cap.read()
        if ret:
            frame = count_vehicles_and_draw(frame, lane_key)
        else:
            frame = np.zeros((300, 400, 3), dtype=np.uint8)
            cv2.putText(
                frame,
                f"No feed for {lane_key.upper()}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 0, 255),
                2,
            )
        cap.release()
        return frame


def display_signals():
    """
    Display the current traffic signal status for all lanes along with timers.
    """
    global next_signal, reason
    signal_image = np.zeros((300, 650, 3), dtype=np.uint8)

    positions = {
        "lane_1": (50, 30),
        "lane_2": (50, 70),
        "lane_3": (50, 110),
        "lane_4": (50, 150),
    }

    colors = {
        "red": (0, 0, 255),
        "green": (0, 255, 0),
        "yellow": (0, 255, 255),
    }

    # Determining the next lane with the highest vehicle count
    next_lane = next_signal
    next_note = f"Next GREEN: {next_lane.upper()} {reason}"
    
    for lane, state in signal_states.items():
        pos = positions[lane]
        color = colors[state]
        timer = signal_timers[lane]
        text = f"{lane.upper()}: {state.upper()} - {timer}s"
        cv2.putText(signal_image, text, (pos[0], pos[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
    # Adding the note for the next green signal
    cv2.putText(signal_image, next_note, (50, 230), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


    cv2.imshow("Traffic Signals", signal_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        global stop_process
        stop_process = True


def transition_to_yellow(lane_key):
    """
    Transition a lane's signal to yellow for 5 seconds.
    """
    global signal_states, signal_timers
    signal_states[lane_key] = "yellow"
    signal_timers[lane_key] = 5
    print(f"🚦{lane_key.upper()} YELLOW for 5 seconds.")
    while signal_timers[lane_key] > 0:
        display_signals()
        time.sleep(1)
        signal_timers[lane_key] -= 1


def manage_signals():
    """
    Manage traffic signals based on vehicle counts.
    """
    global lanes, signal_states, signal_timers, stop_process, next_signal, current_green, reason
    while not stop_process:
        # Sorting lanes by vehicle count and manage signals
        reason="More Vehicles"
        flag=0
        lanes_data=copy.deepcopy(lanes)
        data=list(lanes_data)
        while flag<4:
            vehicount={}
            flag=flag+1
            for lane_key in lanes_data.keys():
                vehicount[lane_key]=lanes[lane_key]["count"]
            lane= max(vehicount, key=vehicount.get)
            next_signal=lane
            reason="More Vehicles"
            vehicle_count = vehicount[lane]
            del lanes_data[lane]
            if vehicle_count > 0:
                green_time = min(max(10, vehicle_count * 2), 60)  # Green time limits

                # Turning other signals to red
                for other_lane in signal_states:
                    if other_lane != lane and signal_states[other_lane] != "red":
                        transition_to_yellow(other_lane)
                        signal_states[other_lane] = "red"

                # Transition from red to green
                if signal_states[lane] == "red":
                    transition_to_yellow(lane)
                    signal_states[lane] = "green"
                    reason=""
                    next_signal=""
                    signal_timers[lane] = green_time
                current_green=lane
                print(f"🚦{lane.upper()} GREEN for {green_time} seconds. Vehicles: {vehicle_count}")
                while signal_timers[lane] > 0:
                    display_signals()
                    time.sleep(1)
                    signal_timers[lane] -= 1
                    prev_time=signal_timers[lane]
                    for lane_key in lanes.keys():
                        if lanes[lane_key]["ambulance_detected"]:
                            next_signal=lane_key
                            reason="Ambulance"
                            prioritize_ambulane_lane(lane_key)
                            if stop_process:
                                return
                            next_signal=lane
                            reason="More Vehicles"
                            transition_to_yellow(lane)
                            current_green=lane
                    signal_timers[lane]=prev_time
                    signal_states[lane] = "green"
                    next_signal=""
                    reason=""
                    if stop_process:
                        return

                if stop_process:
                    return
            else:
                print(f"🚦{lane.upper()} YELLOW for 5 seconds (No vehicles detected).")
                transition_to_yellow(lane)
                signal_states[lane] = "red"

                if stop_process:
                    return

        print("Cycle complete. Starting over...")

def prioritize_ambulane_lane(lane):
    global signal_states, signal_timers, next_signal, current_green, lanes, reason
    if current_green==lane:
        signal_timers[lane]=signal_timers[lane]+30
        lanes[lane]["ambulance_detected"]=False
        return
    else:
        # Turning other signals to red
        for other_lane in signal_states:
            if other_lane != lane and signal_states[other_lane] != "red":
                transition_to_yellow(other_lane)
                signal_states[other_lane] = "red"
        # Transition from red to green
        if signal_states[lane] == "red":
            transition_to_yellow(lane)
            print(f"🚦{lane.upper()} GREEN for ambulance")
            current_green=lane
            signal_states[lane] = "green"
            signal_timers[lane] = 30
            reason=""
            next_signal=""
        while signal_timers[lane] > 0:
            display_signals()
            time.sleep(1)
            signal_timers[lane] -= 1
            if stop_process:
                return
        lanes[lane]["ambulance_detected"]=False
        transition_to_yellow(lane)
        signal_states[lane]="red"
        signal_timers[lane]=0
        return
        

def detect_ambulance(frame, lane_key, confidence_threshold=0.7):
    if lanes[lane_key]["ambulance_detected"]:
        results = ambulance_model(frame,verbose=False)
        for result in results:
            for box in result.boxes:
                cls_id = int(box.cls.cpu().numpy().item())
                class_name = ambulance_model.names[cls_id]
                if class_name.lower() == "ambulance" and box.conf.cpu().numpy() > confidence_threshold:
                    # Drawing the bounding box on the frame for ambulance
                    x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(
                        frame,
                        "Ambulance",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9,
                        (0, 0, 255),
                        2,
                    )
        return frame
    else:
        results = ambulance_model(frame,verbose=False)
        for result in results:
            for box in result.boxes:
                cls_id = int(box.cls.cpu().numpy().item())
                class_name = ambulance_model.names[cls_id]
                if class_name.lower() == "ambulance" and box.conf.cpu().numpy() > confidence_threshold:
                    lanes[lane_key]["ambulance_detected"] = True
                    print(f"🚑 Ambulance detected in {lane_key}! Prioritizing this lane.")
                    # Drawing the bounding box on the frame for ambulance
                    x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(
                        frame,
                        "Ambulance",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9,
                        (0, 0, 255),
                        2,
                    )
        return frame
    
            
def display_cameras():
    """
    Display all 4 CCTV feeds in one frame.
    """
    global stop_process, caps

    lane_labels = {
        "lane_1": "Lane 1",
        "lane_2": "Lane 2",
        "lane_3": "Lane 3",
        "lane_4": "Lane 4",
    }
    
    while not stop_process:
        frames = []
        for lane_key, cap in caps.items():
            for _ in range(10):
                cap.grab()
            ret, frame = cap.read()
            if ret:
                # Adding lane label to the frame
                label = lane_labels[lane_key]
                cv2.putText(
                    frame,
                    label,
                    (10, 55),  
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,  
                    (0, 255, 255),  
                    4,  
                )
                frame = count_vehicles_and_draw(frame, lane_key)
                frame= detect_ambulance(frame, lane_key)
            else:
                # If video ends or fails, display a placeholder
                frame = np.zeros((300, 400, 3), dtype=np.uint8)
                cv2.putText(
                    frame,
                    "No feed",
                    (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,
                    (0, 0, 255),
                    4,
                )
                stop_process = True
                break
            frames.append(frame)

        # Resizing frames to fit in a single window
        resized_frames = [cv2.resize(f, (400, 300)) for f in frames]
        row1 = np.hstack(resized_frames[:2])  
        row2 = np.hstack(resized_frames[2:])  
        combined_frame = np.vstack([row1, row2])

        cv2.imshow("All CCTV Feeds", combined_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            stop_process = True
            break

    # Release video captures and close windows
    for cap in caps.values():
        cap.release()
    cv2.destroyAllWindows()

def main():
    """
    Main loop to monitor traffic and manage signals.
    """
    global stop_process
    process_lane()
    # Start threads for camera display and signal management
    camera_thread = threading.Thread(target=display_cameras)
    signal_thread = threading.Thread(target=manage_signals)

    camera_thread.start()
    signal_thread.start()

    camera_thread.join()
    signal_thread.join()

    print("Process stopped.")


if __name__ == "__main__":
    main()

🚦LANE_1 YELLOW for 5 seconds.
🚦LANE_1 GREEN for 56 seconds. Vehicles: 28
🚦LANE_1 YELLOW for 5 seconds.
🚦LANE_3 YELLOW for 5 seconds.
🚦LANE_3 GREEN for 60 seconds. Vehicles: 31
🚦LANE_3 YELLOW for 5 seconds.
🚦LANE_4 YELLOW for 5 seconds.
🚦LANE_4 GREEN for 36 seconds. Vehicles: 18
🚦LANE_4 YELLOW for 5 seconds.
🚦LANE_2 YELLOW for 5 seconds.
🚦LANE_2 GREEN for 18 seconds. Vehicles: 9
Cycle complete. Starting over...
🚦LANE_2 YELLOW for 5 seconds.
🚦LANE_1 YELLOW for 5 seconds.
🚦LANE_1 GREEN for 60 seconds. Vehicles: 30


In [ ]:
'''With Ambulance'''
import cv2
from ultralytics import YOLO
import numpy as np
import time
import copy
import threading

# Loading YOLOv8 model
model = YOLO("yolov8n.pt") 

#Loading ambulance trained model 
ambulance_model = YOLO("C:/Users/ragha/runs/detect/train5/weights/best.pt")

# Defining vehicle classes (COCO dataset IDs for vehicles)
vehicle_classes = [2, 3, 5, 7]  # Car, Bus, Truck, Motorcycle

# Initializing variables for lane monitoring
lanes = {
    "lane_1": {"cctv": "A1.mp4", "count": 0, "ambulance_detected": False},
    "lane_2": {"cctv": "A2.mp4", "count": 0, "ambulance_detected": False},
    "lane_3": {"cctv": "Lane3.mp4", "count": 0, "ambulance_detected": False},
    "lane_4": {"cctv": "Lane4.mp4", "count": 0, "ambulance_detected": False}
}

# Signal states and timers
signal_states = {"lane_1": "red", "lane_2": "red", "lane_3": "red", "lane_4": "red"}
signal_timers = {"lane_1": 0, "lane_2": 0, "lane_3": 0, "lane_4": 0}

# Flag to stop the process
stop_process = False
lock = threading.Lock()


caps = {lane_key: cv2.VideoCapture(lanes[lane_key]["cctv"]) for lane_key in lanes.keys()}
for cap in caps.values():
        cap.set(cv2.CAP_PROP_FPS, 1200)

def count_vehicles_and_draw(frame, lane_key):
    """
    Detect and count vehicles in the frame using YOLO, and draw bounding boxes.
    """
    results = model(frame, stream=False, verbose=False)
    vehicle_count = 0

    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls.cpu().numpy().item())
            if cls_id in vehicle_classes:  # Checking if the detected object is a vehicle
                vehicle_count += 1

                # Drawing bounding box
                x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    frame,
                    model.names[cls_id],
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    2,
                )

    with lock:
        lanes[lane_key]["count"] = vehicle_count
    return frame


def process_lane():
    """
    Process each lane's CCTV feed and update vehicle counts with detection frames.
    """
    for lane_key in ['lane_1','lane_2','lane_3', 'lane_4']:
        cap =cv2.VideoCapture(lanes[lane_key]["cctv"])
        ret, frame = cap.read()
        if ret:
            frame = count_vehicles_and_draw(frame, lane_key)
        else:
            frame = np.zeros((300, 400, 3), dtype=np.uint8)
            cv2.putText(
                frame,
                f"No feed for {lane_key.upper()}",
                (50, 150),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 0, 255),
                2,
            )
        cap.release()
        return frame


def display_signals():
    """
    Display the current traffic signal status for all lanes along with timers.
    """
    global next_signal, reason
    signal_image = np.zeros((300, 650, 3), dtype=np.uint8)

    positions = {
        "lane_1": (50, 30),
        "lane_2": (50, 70),
        "lane_3": (50, 110),
        "lane_4": (50, 150),
    }

    colors = {
        "red": (0, 0, 255),
        "green": (0, 255, 0),
        "yellow": (0, 255, 255),
    }

    # Determining the next lane with the highest vehicle count
    next_lane = next_signal
    next_note = f"Next GREEN: {next_lane.upper()} {reason}"
    
    for lane, state in signal_states.items():
        pos = positions[lane]
        color = colors[state]
        timer = signal_timers[lane]
        text = f"{lane.upper()}: {state.upper()} - {timer}s"
        cv2.putText(signal_image, text, (pos[0], pos[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
    # Adding the note for the next green signal
    cv2.putText(signal_image, next_note, (50, 230), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


    cv2.imshow("Traffic Signals", signal_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        global stop_process
        stop_process = True


def transition_to_yellow(lane_key):
    """
    Transition a lane's signal to yellow for 5 seconds.
    """
    global signal_states, signal_timers
    signal_states[lane_key] = "yellow"
    signal_timers[lane_key] = 5
    print(f"🚦{lane_key.upper()} YELLOW for 5 seconds.")
    while signal_timers[lane_key] > 0:
        display_signals()
        time.sleep(1)
        signal_timers[lane_key] -= 1


def manage_signals():
    """
    Manage traffic signals based on vehicle counts.
    """
    global lanes, signal_states, signal_timers, stop_process, next_signal, current_green, reason
    while not stop_process:
        # Sorting lanes by vehicle count and manage signals
        reason="More Vehicles"
        flag=0
        lanes_data=copy.deepcopy(lanes)
        data=list(lanes_data)
        while flag<4:
            vehicount={}
            flag=flag+1
            for lane_key in lanes_data.keys():
                vehicount[lane_key]=lanes[lane_key]["count"]
            lane= max(vehicount, key=vehicount.get)
            next_signal=lane
            reason="More Vehicles"
            vehicle_count = vehicount[lane]
            del lanes_data[lane]
            if vehicle_count > 0:
                green_time = min(max(10, vehicle_count * 2), 60)  # Green time limits

                # Turning other signals to red
                for other_lane in signal_states:
                    if other_lane != lane and signal_states[other_lane] != "red":
                        transition_to_yellow(other_lane)
                        signal_states[other_lane] = "red"

                # Transition from red to green
                if signal_states[lane] == "red":
                    transition_to_yellow(lane)
                    signal_states[lane] = "green"
                    reason=""
                    next_signal=""
                    signal_timers[lane] = green_time
                current_green=lane
                print(f"🚦{lane.upper()} GREEN for {green_time} seconds. Vehicles: {vehicle_count}")
                while signal_timers[lane] > 0:
                    display_signals()
                    time.sleep(1)
                    signal_timers[lane] -= 1
                    prev_time=signal_timers[lane]
                    for lane_key in lanes.keys():
                        if lanes[lane_key]["ambulance_detected"]:
                            next_signal=lane_key
                            reason="Ambulance"
                            prioritize_ambulane_lane(lane_key)
                            if stop_process:
                                return
                            next_signal=lane
                            reason="More Vehicles"
                            transition_to_yellow(lane)
                            current_green=lane
                    signal_timers[lane]=prev_time
                    signal_states[lane] = "green"
                    next_signal=""
                    reason=""
                    if stop_process:
                        return

                if stop_process:
                    return
            else:
                print(f"🚦{lane.upper()} YELLOW for 5 seconds (No vehicles detected).")
                transition_to_yellow(lane)
                signal_states[lane] = "red"

                if stop_process:
                    return

        print("Cycle complete. Starting over...")

def prioritize_ambulane_lane(lane):
    global signal_states, signal_timers, next_signal, current_green, lanes, reason
    if current_green==lane:
        signal_timers[lane]=signal_timers[lane]+30
        lanes[lane]["ambulance_detected"]=False
        return
    else:
        # Turning other signals to red
        for other_lane in signal_states:
            if other_lane != lane and signal_states[other_lane] != "red":
                transition_to_yellow(other_lane)
                signal_states[other_lane] = "red"
        # Transition from red to green
        if signal_states[lane] == "red":
            transition_to_yellow(lane)
            print(f"🚦{lane.upper()} GREEN for ambulance")
            current_green=lane
            signal_states[lane] = "green"
            signal_timers[lane] = 30
            reason=""
            next_signal=""
        while signal_timers[lane] > 0:
            display_signals()
            time.sleep(1)
            signal_timers[lane] -= 1
            if stop_process:
                return
        lanes[lane]["ambulance_detected"]=False
        transition_to_yellow(lane)
        signal_states[lane]="red"
        signal_timers[lane]=0
        return
        

def detect_ambulance(frame, lane_key, confidence_threshold=0.7):
    if lanes[lane_key]["ambulance_detected"]:
        results = ambulance_model(frame,verbose=False)
        for result in results:
            for box in result.boxes:
                cls_id = int(box.cls.cpu().numpy().item())
                class_name = ambulance_model.names[cls_id]
                if class_name.lower() == "ambulance" and box.conf.cpu().numpy() > confidence_threshold:
                    # Drawing the bounding box on the frame for ambulance
                    x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(
                        frame,
                        "Ambulance",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9,
                        (0, 0, 255),
                        2,
                    )
        return frame
    else:
        results = ambulance_model(frame,verbose=False)
        for result in results:
            for box in result.boxes:
                cls_id = int(box.cls.cpu().numpy().item())
                class_name = ambulance_model.names[cls_id]
                if class_name.lower() == "ambulance" and box.conf.cpu().numpy() > confidence_threshold:
                    lanes[lane_key]["ambulance_detected"] = True
                    print(f"🚑 Ambulance detected in {lane_key}! Prioritizing this lane.")
                    # Drawing the bounding box on the frame for ambulance
                    x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(
                        frame,
                        "Ambulance",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9,
                        (0, 0, 255),
                        2,
                    )
        return frame
    
            
def display_cameras():
    """
    Display all 4 CCTV feeds in one frame.
    """
    global stop_process, caps

    lane_labels = {
        "lane_1": "Lane 1",
        "lane_2": "Lane 2",
        "lane_3": "Lane 3",
        "lane_4": "Lane 4",
    }
    
    while not stop_process:
        frames = []
        for lane_key, cap in caps.items():
            for _ in range(10):
                cap.grab()
            ret, frame = cap.read()
            if ret:
                # Adding lane label to the frame
                label = lane_labels[lane_key]
                cv2.putText(
                    frame,
                    label,
                    (10, 55),  
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,  
                    (0, 255, 255),  
                    4,  
                )
                frame = count_vehicles_and_draw(frame, lane_key)
                frame= detect_ambulance(frame, lane_key)
            else:
                # If video ends or fails, display a placeholder
                frame = np.zeros((300, 400, 3), dtype=np.uint8)
                cv2.putText(
                    frame,
                    "No feed",
                    (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,
                    (0, 0, 255),
                    4,
                )
                stop_process = True
                break
            frames.append(frame)

        # Resizing frames to fit in a single window
        resized_frames = [cv2.resize(f, (400, 300)) for f in frames]
        row1 = np.hstack(resized_frames[:2])  
        row2 = np.hstack(resized_frames[2:])  
        combined_frame = np.vstack([row1, row2])

        cv2.imshow("All CCTV Feeds", combined_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            stop_process = True
            break

    # Release video captures and close windows
    for cap in caps.values():
        cap.release()
    cv2.destroyAllWindows()

def main():
    """
    Main loop to monitor traffic and manage signals.
    """
    global stop_process
    process_lane()
    # Start threads for camera display and signal management
    camera_thread = threading.Thread(target=display_cameras)
    signal_thread = threading.Thread(target=manage_signals)

    camera_thread.start()
    signal_thread.start()

    camera_thread.join()
    signal_thread.join()

    print("Process stopped.")


if __name__ == "__main__":
    main()

🚦LANE_1 YELLOW for 5 seconds.
🚑 Ambulance detected in lane_2! Prioritizing this lane.
🚦LANE_1 GREEN for 58 seconds. Vehicles: 29
🚦LANE_1 YELLOW for 5 seconds.
🚦LANE_2 YELLOW for 5 seconds.
🚦LANE_2 GREEN for ambulance
